In [3]:
import duckdb
import os

In [6]:
try:
    # Delete the secret if it exists
    duckdb.sql("DROP SECRET IF EXISTS secret1")
except Exception as e:
    print(e)

duckdb.sql(
    f"""
CREATE SECRET secret1 (
    TYPE S3,
    KEY_ID '{os.getenv('AWS_ACCESS_KEY_ID')}',
    SECRET '{os.getenv('AWS_SECRET_ACCESS_KEY')}',
    ENDPOINT '{os.getenv('AWS_S3_ENDPOINT')}'
);
    """
)

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

In [8]:
duckdb.sql(
    """
CREATE TABLE forecast_lv_grid AS
    SELECT
        *
    FROM 's3://md-raw-data/*/forecast_lv_grid.parquet'
    """
)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [9]:
duckdb.sql(
    """
CREATE TABLE current_weather_lv_grid AS
    SELECT
        *
    FROM 's3://md-raw-data/*/current_weather_lv_grid.parquet'
    """
)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [12]:
display(duckdb.sql("SELECT * FROM forecast_lv_grid"))
duckdb.sql("SELECT * FROM current_weather_lv_grid")

┌───────────────────────────────┬──────────────────────────┬────────────────┬───────────────────┬────────────────────┬───────────┬─────────────┬────────────┬────────────────┬───────────────────┬─────────────┬──────────────────────┬───────────────┬───────────────────────────┬───────┬─────────┬──────────┬────────────┬──────────────┬─────────────┬─────────────────┬─────────────────┬──────────────────┐
│          created_at           │           date           │     model      │     latitude      │     longitude      │ elevation │ temperature │ wind_speed │ wind_direction │ relative_humidity │  dew_point  │ apparent_temperature │ precipitation │ precipitation_probability │ rain  │ showers │ snowfall │ snow_depth │ weather_code │ cloud_cover │ cloud_cover_low │ cloud_cover_mid │ cloud_cover_high │
│   timestamp with time zone    │ timestamp with time zone │    varchar     │      double       │       double       │  double   │    float    │   float    │     float      │       float       │  

┌───────────────────────────────┬──────────────────────────┬──────────────────────┬────────────────────┬────────────────────┬───────────┬─────────────────────┬────────────────────┬────────────────────┬───────────────────┬───────────────────────┬──────────────────────┬─────────────────────┬───────────────────────────┬─────────────────────┬─────────┬──────────┬──────────────────────┬──────────────┬─────────────┬───────────────────┬────────────────────┬──────────────────┐
│          created_at           │           date           │        model         │      latitude      │     longitude      │ elevation │     temperature     │     wind_speed     │   wind_direction   │ relative_humidity │       dew_point       │ apparent_temperature │    precipitation    │ precipitation_probability │        rain         │ showers │ snowfall │      snow_depth      │ weather_code │ cloud_cover │  cloud_cover_low  │  cloud_cover_mid   │ cloud_cover_high │
│   timestamp with time zone    │ timestamp with tim

In [18]:
weather_variables = [
        "temperature",
        "wind_speed",
        "wind_direction",
        "relative_humidity",
        "dew_point",
        "apparent_temperature",
        "precipitation",
        "precipitation_probability",
        "rain",
        "showers",
        "snowfall",
        "snow_depth",
        "weather_code",
        "cloud_cover",
        "cloud_cover_low",
        "cloud_cover_mid",
        "cloud_cover_high",
]

diff_statements = [
    f"abs(forecast_lv_grid.{variable} - current_weather_lv_grid.{variable}) AS {variable}_difference"
    for variable in weather_variables
]


In [19]:
duckdb.sql(
    f"""
SELECT
    forecast_lv_grid.date - forecast_lv_grid.created_at AS forecast_lead_time,
    {", ".join(diff_statements)}
FROM forecast_lv_grid
JOIN current_weather_lv_grid ON
    forecast_lv_grid.date = current_weather_lv_grid.date
    AND forecast_lv_grid.latitude = current_weather_lv_grid.latitude
    AND forecast_lv_grid.longitude = current_weather_lv_grid.longitude
    """
)


┌────────────────────────┬────────────────────────┬───────────────────────┬───────────────────────────┬──────────────────────────────┬──────────────────────┬─────────────────────────────────┬──────────────────────────┬──────────────────────────────────────┬─────────────────────┬────────────────────┬─────────────────────┬───────────────────────┬─────────────────────────┬────────────────────────┬────────────────────────────┬────────────────────────────┬─────────────────────────────┐
│   forecast_lead_time   │ temperature_difference │ wind_speed_difference │ wind_direction_difference │ relative_humidity_difference │ dew_point_difference │ apparent_temperature_difference │ precipitation_difference │ precipitation_probability_difference │   rain_difference   │ showers_difference │ snowfall_difference │ snow_depth_difference │ weather_code_difference │ cloud_cover_difference │ cloud_cover_low_difference │ cloud_cover_mid_difference │ cloud_cover_high_difference │
│        interval        │  